# Odds & Sods


In [ ]:
subject_results = pd.concat([subject, preprocessing_results, group_summary])
# Flatten all the channels
for idx, row in specparam_df.iterrows():
    row_series = pd.Series(row.values, index=[f'chn_{idx}_{col}' for col in row.index])
    subject_results = pd.concat([subject_results, row_series])

In [ ]:
def check_nans(data, nan_policy='zero'):
    """Check an array for nan values, and replace, based on policy."""

    # Find where there are nan values in the data
    nan_inds = np.where(np.isnan(data))

    # Apply desired nan policy to data
    if nan_policy == 'zero':
        data[nan_inds] = 0
    elif nan_policy == 'mean':
        data[nan_inds] = np.nanmean(data)
    else:
        raise ValueError('Nan policy not understood.')

    return data

## SPM Get Bands

In [ ]:
from specparam import Bands
from specparam.data.periodic import get_band_peak, get_band_peak_group

bands = Bands({'Delta': [0.5, 4],
               'Theta': [4, 8],
               'Alpha': [8, 13],
               'Beta': [13, 30],
               'Gamma': [30, 100]
               })

deltas = get_band_peak_group(fg, bands.Delta)
thetas = get_band_peak_group(fg, bands.Theta)
alphas = get_band_peak_group(fg, bands.Alpha)
betas = get_band_peak_group(fg, bands.Beta)
gammas = get_band_peak_group(fg, bands.Gamma)



In [ ]:
# # Plot test channels - Bands

# from specparam import Bands
# from specparam.data.periodic import get_band_peak, get_band_peak_group

# bands = Bands({'Delta': [0.5, 4],
#                'Theta': [4, 8],
#                'Alpha': [8, 13],
#                'Beta': [13, 30],
#                'Gamma': [30, 100]
#                })

# for channel_name in ['F4']:
#     channel_idx = psd.ch_names.index(channel_name)
#     fm = fg.get_model(ind=channel_idx, regenerate=True)

#     all_bands = []
#     for band_label in bands.labels:
#         band_range = bands[band_label]
#         peak_details = (band_label, get_band_peak(fm, band_range, select_highest=False))
#         print(peak_details)
        
#         # Ensure peak_details[1] is always a 2D array (n_peaks, 3)
#         # peak_arr = np.atleast_2d(peak_details[1])
#         # peak_details = (peak_details[0], peak_arr)
#         # all_bands.append(peak_details)
#     # plot_spm_fit_withbands(fm, channel_name, all_bands)
#     # print(all_bands)

#     # alpha_peak_max = get_band_peak(fm, bands['Alpha'], select_highest=True)

#     # # Get the label for the alpha band
#     # alpha_label = [label for label in bands.labels if bands[label] == bands['Alpha']][0]
#     # print(f"Band label for bands['Alpha']: {alpha_label}")

#     # plot_spm_fit_withbands(fm, channel_name, alpha_peak_max[0] )
#     # print(alpha_peak_max)




In [ ]:
# Function to plot a SpecParam Fit - With Bands
#

def plot_spm_fit_withbands(fm, channel_name, all_bands):

    print(f'SpecParam for Channel: {channel_name}')
    fm.print_results()

    freqs = fm.freqs
    flattened_data = fm.get_model(component='peak', space='log')

    # Three plots
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))
    fig.suptitle(f'SpecParam Fit for Channel: {channel_name}', fontsize=20)
    axes[0].set_title('Fit + Peaks')
    fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'}, ax=axes[0])

    print(all_bands)
    # for band_label, band_peak in all_bands:
    #     if band_peak is not None and not np.isnan(band_peak[0]):
    #         # axes[0].axvline(x=band_peak[0], color='skyblue', linestyle='--', linewidth=1.5, label=band_label)
    #         # axes[0].axvline(x=band_peak[0], color=plt.cm.tab10(bands.labels.index(band_label)), linestyle='--', linewidth=2, label=band_label)
    #         color = plt.get_cmap('tab10')(bands.labels.index(band_label))
    #         axes[0].axvline(x=band_peak[0], color=color, linestyle='--', linewidth=2, label=band_label)
    #         axes[0].legend()
    # handles, labels = axes[0].get_legend_handles_labels()
    # by_label = dict(zip(labels, handles))
    # axes[0].legend(by_label.values(), by_label.keys())

    for band_label, band_peak in all_bands:
        for peak in band_peak:
            print(band_label, peak)


        #     # Handle if band_peak is an array of multiple peaks (shape (N, 3))
        # if isinstance(band_peak, np.ndarray) and band_peak.ndim == 2:
        #     for peak in band_peak:
        #     if peak is not None and not np.isnan(peak[0]):
        #         color = plt.get_cmap('tab10')(bands.labels.index(band_label))
        #         axes[0].axvline(x=peak[0], color=color, linestyle='--', linewidth=2, label=band_label)
        #     axes[0].legend()
        #     continue


    axes[1].set_title('Flattened - Offset Removed')
    axes[1].plot(freqs, flattened_data)
    axes[2].set_title('Log/Log + Peaks')
    fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'}, plt_log=True, ax=axes[2])
    plt.show()
    

### Compares Scores ??

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score

def evaluate_model(model, X, y, cv_folds=5):
    """Comprehensive model evaluation"""
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Multiple metrics
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
    results = {}
    
    for metric in scoring:
        scores = cross_val_score(model, X, y, cv=skf, scoring=metric)
        results[metric] = {
            'mean': np.mean(scores),
            'std': np.std(scores),
            'scores': scores
        }
    
    return results

evaluate_model(model_randforest, X_test_transformed, y_test)